In [1]:
!pip -q install gspread gspread-dataframe google-auth


In [2]:
import gspread
from google.colab import auth
from google.auth import default
from gspread_dataframe import set_with_dataframe


In [3]:
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
import warnings
warnings.filterwarnings("ignore")

In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from statsmodels.tsa.holtwinters import ExponentialSmoothing

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [5]:
SHEET_ID = "1Kj3vDGxZZsjd-zBqMWTSR_xC-eeuTmlvKK6MWIL3pBg"

sh = gc.open_by_key(SHEET_ID)
raw_ws = sh.worksheet("Raw_Data")

df = pd.DataFrame(raw_ws.get_all_records())
df.columns = df.columns.str.strip()

print(df.shape)
df.head()


(25719, 66)


,Title,Authors,Number of Authors,Scopus Author Ids,Year,Full date,Scopus Source title,Volume,Issue,Pages,...,ANZSRC FoR (2020) code,ANZSRC FoR (2020) name,Sustainable Development Goals (2023),Topic Cluster name,Topic Cluster number,Topic Cluster Prominence Percentile,Topic name,Topic number,Topic Prominence Percentile,Publication link to Topic strength
0,Global burden of 369 diseases and injuries in ...,"Abbafati, C.| Abbas, K.M.| Abbasi, M.| Abbasif...",1598,54917122400| 57190285707| 57221004192| 3705340...,2020,2020-10-17,The Lancet,396,10258,1204-1222,...,-,-,SDG 3| SDG 16,COVID-19; Severe Acute Respiratory Syndrome Co...,1,99.935,Global Disease Burden; Prevalence; COVID-19,39350,96.222,Other
1,Global Burden of Cardiovascular Diseases and R...,"Roth, G.A.| Mensah, G.A.| Johnson, C.O.| Addol...",610,55074747400| 57196028147| 57212301236| 7006562...,2020,2020-12-22,Journal of the American College of Cardiology,76,25,2982-3021,...,3201,Cardiovascular medicine and haematology,SDG 3| SDG 4| SDG 12,Cardiovascular System; Sodium; Randomized Cont...,18,87.508,Blood Pressure; Prevalence; Public Health,13187,98.082,Other
2,World Health Organization 2020 guidelines on p...,"Bull, F.C.| Al-Ansari, S.S.| Biddle, S.| Borod...",30,7006165716| 58825729400| 57203868033| 82256396...,2020,2020-12-01,British Journal of Sports Medicine,54,24,1451-1462,...,3202| 5201| 4207,Clinical sciences| Applied and developmental p...,SDG 3,Physical Activity; Health Promotion; Quality o...,445,77.763,Sedentary Behavior; Accelerometry; Physical Ac...,2405,98.624,Defensible
3,"Global, regional, and national burden of strok...","Feigin, V.L.| Stark, B.A.| Johnson, C.O.| Roth...",417,57203677957| 57219860666| 57212301236| 5507474...,2021,2021-01-01,The Lancet Neurology,20,10,1-26,...,3209| 3202,Neurosciences| Clinical sciences,SDG 3,Ischemic Stroke; Thrombectomy; Computed Tomogr...,113,86.266,Apoplexy; Brain Ischemia; Ischemic Stroke,3648,97.879,Defensible
4,"Global, regional, and national burden of neuro...","Feigin, V.L.| Nichols, E.| Alam, T.| Bannick, ...",510,57203677957| 57205756163| 57205589820| 5720683...,2019,2019-05-01,The Lancet Neurology,18,5,459-480,...,3202| 3209,Clinical sciences| Neurosciences,SDG 3,Auras; Quality of Life; Randomized Controlled ...,468,64.16,Migraine; Monoclonal Antibody; Randomized Cont...,9257,98.865,Defensible


In [6]:
# =========================
# 0) CONFIG (Google Drive file)
# =========================
#from google.colab import drive
#drive.mount('/content/drive')

#FILE_PATH = "/content/drive/MyDrive/Research folder/Msc Research/Publications_in_Sri_Lanka_2019_-_2025.xlsx"

TARGET_COL = "Publications"

TRAIN_YEARS = [2019, 2020, 2021, 2022]
VAL_YEARS   = [2023]
TEST_YEARS  = [2024]
EXCLUDE_YEARS = [2025]

FORECAST_YEARS = [2026, 2027, 2028]

TOP_FIELDS = 30


In [7]:
#df = pd.read_excel(FILE_PATH, sheet_name=0)


In [8]:
#df.head()

In [9]:
# =========================
# 2) KEEP REQUIRED COLUMNS (include IDs to count unique papers)
# =========================
needed = [
    "Year",
    "Citations",
    "Field-Weighted Citation Impact",
    "Number of Authors",
    "Number of Institutions",
    "Number of Countries/Regions",
    "All Science Journal Classification (ASJC) field name",
    "EID",
    "DOI",
]
needed = [c for c in needed if c in df.columns]
df = df[needed].copy()

In [10]:
# =========================
# 3) BASIC CLEANING
# =========================
df = df.dropna(subset=["Year"])
df["Year"] = pd.to_numeric(df["Year"], errors="coerce")
df = df.dropna(subset=["Year"])
df["Year"] = df["Year"].astype(int)

# Exclude partial year (2025)
df = df[~df["Year"].isin(EXCLUDE_YEARS)].copy()

# Ensure numeric for numeric columns
num_cols = [
    "Citations",
    "Field-Weighted Citation Impact",
    "Number of Authors",
    "Number of Institutions",
    "Number of Countries/Regions",
]
for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")


In [11]:

# =========================
# 4) EXPLODE ASJC FIELD NAMES (split by "|")
# =========================
field_col = "All Science Journal Classification (ASJC) field name"
df[field_col] = df[field_col].fillna("Unknown").astype(str)
df[field_col] = df[field_col].apply(
    lambda x: [s.strip() for s in x.split("|")] if "|" in x else [x.strip()]
)

df = df.explode(field_col).rename(columns={field_col: "ASJC_Field"})
df["ASJC_Field"] = df["ASJC_Field"].replace("", "Unknown")

# Keep only top fields to reduce fragmentation
top_fields = df["ASJC_Field"].value_counts().head(TOP_FIELDS).index.tolist()
df = df[df["ASJC_Field"].isin(top_fields)].copy()

In [12]:
# =========================
# 5) AGGREGATE TO FIELD x YEAR
# =========================
agg_dict = {
    "Citations": "sum",
    "Field-Weighted Citation Impact": "mean",
    "Number of Authors": "mean",
    "Number of Institutions": "mean",
    "Number of Countries/Regions": "mean",
}
agg_dict = {k: v for k, v in agg_dict.items() if k in df.columns}
panel = (
    df.groupby(["ASJC_Field", "Year"])
      .agg(**{k: (k, v) for k, v in agg_dict.items()})
      .reset_index()
)

# Publications count: use unique paper ID (EID preferred, DOI fallback)
if "EID" in df.columns and df["EID"].notna().any():
    id_col = "EID"
elif "DOI" in df.columns and df["DOI"].notna().any():
    id_col = "DOI"
else:
    id_col = None

if id_col:
    pub_counts = (
        df.groupby(["ASJC_Field", "Year"])[id_col]
          .nunique()
          .reset_index(name="Publications")
    )
else:
    pub_counts = (
        df.groupby(["ASJC_Field", "Year"])
          .size()
          .reset_index(name="Publications")
    )

panel = panel.merge(pub_counts, on=["ASJC_Field", "Year"], how="left")

# Fill missing values
for c in panel.columns:
    if c not in ["ASJC_Field", "Year"]:
        panel[c] = panel[c].fillna(0)

# Sanity checks
print("Panel shape:", panel.shape)
print("Years in panel:", sorted(panel["Year"].unique()))
print(panel.head())

Panel shape: (180, 8)
Years in panel: [np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]
                ASJC_Field  Year  Citations  Field-Weighted Citation Impact  \
0  Artificial Intelligence  2019       2575                        1.103029   
1  Artificial Intelligence  2020       2524                        0.964317   
2  Artificial Intelligence  2021       1740                        0.726475   
3  Artificial Intelligence  2022       2778                        0.866236   
4  Artificial Intelligence  2023       1059                        0.591563   

   Number of Authors  Number of Institutions  Number of Countries/Regions  \
0           3.742627                1.222520                     1.193029   
1           3.914634                1.221951                     1.187805   
2           4.113082                1.150776                     1.175166   
3           4.191882                1.284133                     1.252768   
4    

In [13]:
# =========================
# 6) ETS BASELINE PER FIELD (train on TRAIN_YEARS)
# =========================
def fit_ets_and_predict(field_df: pd.DataFrame, years_to_predict: list[int]):
    """
    Fit ETS on TRAIN_YEARS for one field, return:
    - preds for years_to_predict
    - fitted values for train years
    """
    field_df = field_df.sort_values("Year")
    series = field_df.set_index("Year")["Publications"]
    train_series = series.loc[series.index.isin(TRAIN_YEARS)]

    # Too few points => fallback
    if train_series.shape[0] < 3:
        last_val = float(train_series.iloc[-1]) if train_series.shape[0] > 0 else 0.0
        preds = {y: last_val for y in years_to_predict}
        fitted = {y: last_val for y in TRAIN_YEARS}
        return preds, fitted

    try:
        model = ExponentialSmoothing(
            train_series,
            trend="add",
            damped_trend=True,
            seasonal=None,
            initialization_method="estimated"
        ).fit(optimized=True)

        fitted_train = model.fittedvalues.to_dict()

        # Forecast until max(years_to_predict)
        steps = max(years_to_predict) - max(TRAIN_YEARS)
        forecast_vals = model.forecast(steps=steps)

        preds = {}
        for y in years_to_predict:
            if y in fitted_train:  # train years
                preds[y] = float(fitted_train[y])
            else:
                preds[y] = float(forecast_vals.loc[y])
        return preds, fitted_train

    except Exception:
        last_val = float(train_series.iloc[-1])
        preds = {y: last_val for y in years_to_predict}
        fitted = {y: last_val for y in TRAIN_YEARS}
        return preds, fitted

# Build baseline predictions for train/val/test
baseline_rows = []
years_needed = TRAIN_YEARS + VAL_YEARS + TEST_YEARS

for field, g in panel.groupby("ASJC_Field"):
    preds, fitted_train = fit_ets_and_predict(g, years_needed)
    for y in years_needed:
        baseline_rows.append((field, y, float(preds[y])))

baseline = pd.DataFrame(baseline_rows, columns=["ASJC_Field", "Year", "ETS_Pred"])
panel2 = panel.merge(baseline, on=["ASJC_Field", "Year"], how="inner")

# Residual = actual - ETS baseline
panel2["Residual"] = panel2["Publications"] - panel2["ETS_Pred"]

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
 

In [14]:
# =========================
# 7) TRAIN / VAL / TEST SPLIT (chronological)
# =========================
train_df = panel2[panel2["Year"].isin(TRAIN_YEARS)].copy()
val_df   = panel2[panel2["Year"].isin(VAL_YEARS)].copy()
test_df  = panel2[panel2["Year"].isin(TEST_YEARS)].copy()

In [15]:
# =========================
# 8) ML MODEL FOR RESIDUALS
# =========================
feature_cols_num = [c for c in [
    "Year",
    "Citations",
    "Field-Weighted Citation Impact",
    "Number of Authors",
    "Number of Institutions",
    "Number of Countries/Regions",
] if c in panel2.columns]

feature_cols_cat = ["ASJC_Field"]

X_train = train_df[feature_cols_cat + feature_cols_num]
y_train = train_df["Residual"]

X_val = val_df[feature_cols_cat + feature_cols_num]
y_val = val_df["Residual"]

X_test = test_df[feature_cols_cat + feature_cols_num]
y_test = test_df["Residual"]

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), feature_cols_cat),
        ("num", "passthrough", feature_cols_num),
    ]
)

residual_model = Pipeline(
    steps=[
        ("prep", preprocess),
        ("model", RandomForestRegressor(
            n_estimators=400,
            random_state=42,
            max_depth=None,
            min_samples_leaf=2,
            n_jobs=-1
        ))
    ]
)

residual_model.fit(X_train, y_train)

# Predict residuals
val_df["Residual_Pred"]  = residual_model.predict(X_val)
test_df["Residual_Pred"] = residual_model.predict(X_test)

# Hybrid prediction
val_df["Hybrid_Pred"]  = val_df["ETS_Pred"] + val_df["Residual_Pred"]
test_df["Hybrid_Pred"] = test_df["ETS_Pred"] + test_df["Residual_Pred"]

# ETS-only baseline
val_df["ETS_Only_Pred"]  = val_df["ETS_Pred"]
test_df["ETS_Only_Pred"] = test_df["ETS_Pred"]

# ML-only comparison (predict Publications directly)
direct_model = Pipeline(
    steps=[
        ("prep", preprocess),
        ("model", RandomForestRegressor(
            n_estimators=400,
            random_state=42,
            min_samples_leaf=2,
            n_jobs=-1
        ))
    ]
)

direct_model.fit(X_train, train_df["Publications"])
val_df["ML_Only_Pred"]  = direct_model.predict(X_val)
test_df["ML_Only_Pred"] = direct_model.predict(X_test)

In [16]:
# =========================
# 9) EVALUATION
# =========================
def report_metrics(y_true, y_pred, label):
    mae  = mean_absolute_error(y_true, y_pred)
    mse  = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)   # ← manual RMSE (fix)
    r2   = r2_score(y_true, y_pred)
    return {"Model": label, "MAE": mae, "RMSE": rmse, "R2": r2}

results = []
results.append(report_metrics(val_df["Publications"],  val_df["ETS_Only_Pred"], "ETS only (VAL)"))
results.append(report_metrics(val_df["Publications"],  val_df["ML_Only_Pred"],  "ML only (VAL)"))
results.append(report_metrics(val_df["Publications"],  val_df["Hybrid_Pred"],   "Hybrid (VAL)"))

results.append(report_metrics(test_df["Publications"], test_df["ETS_Only_Pred"], "ETS only (TEST)"))
results.append(report_metrics(test_df["Publications"], test_df["ML_Only_Pred"],  "ML only (TEST)"))
results.append(report_metrics(test_df["Publications"], test_df["Hybrid_Pred"],   "Hybrid (TEST)"))
evaluation_df = pd.DataFrame(results)

print("\n=== Field-level Evaluation ===")
print(pd.DataFrame(results))

def national_total(df_year):
    return df_year.groupby("Year")[["Publications", "ETS_Only_Pred", "ML_Only_Pred", "Hybrid_Pred"]].sum()

print("\n=== National totals (Validation year) ===")
print(national_total(val_df))

print("\n=== National totals (Test year) ===")
print(national_total(test_df))


=== Field-level Evaluation ===
             Model        MAE        RMSE        R2
0   ETS only (VAL)  45.900000   65.467295  0.767237
1    ML only (VAL)  82.439877  114.972384  0.282120
2     Hybrid (VAL)  44.790499   61.905598  0.791875
3  ETS only (TEST)  47.600000   71.689609  0.694640
4   ML only (TEST)  95.696708  133.154361 -0.053441
5    Hybrid (TEST)  46.826699   69.090088  0.716384

=== National totals (Validation year) ===
      Publications  ETS_Only_Pred  ML_Only_Pred  Hybrid_Pred
Year                                                        
2023          6632         6155.0   4594.762934  5912.815191

=== National totals (Test year) ===
      Publications  ETS_Only_Pred  ML_Only_Pred  Hybrid_Pred
Year                                                        
2024          6563         6155.0   4139.234391  5686.153189


In [17]:
# =========================
# 10) FORECAST 2026–2028 (ETS on 2019–2024 + ML residual using 2024 features)
# =========================
# Use last available complete year (2024) features per field for future rows
last_year = max(TEST_YEARS)
last_feats = panel2[panel2["Year"] == last_year].copy()

# Build future feature rows
future_rows = []
for field in top_fields:
    base = last_feats[last_feats["ASJC_Field"] == field]
    if base.empty:
        continue
    base = base.iloc[0].to_dict()

    for y in FORECAST_YEARS:
        row = {"ASJC_Field": field, "Year": y}
        for c in feature_cols_num:
            if c == "Year":
                continue
            row[c] = float(base.get(c, 0.0))
        future_rows.append(row)

future_X = pd.DataFrame(future_rows)

# ETS forecasts for 2026–2028 per field (fit on ALL complete years 2019–2024)
ets_future_rows = []
for field, g in panel.groupby("ASJC_Field"):
    g = g.sort_values("Year")
    series = g.set_index("Year")["Publications"]

    if series.shape[0] < 3:
        last_val = float(series.iloc[-1]) if series.shape[0] else 0.0
        for y in FORECAST_YEARS:
            ets_future_rows.append((field, y, last_val))
        continue

    try:
        model = ExponentialSmoothing(
            series,
            trend="add",
            damped_trend=True,
            seasonal=None,
            initialization_method="estimated"
        ).fit(optimized=True)

        steps = max(FORECAST_YEARS) - series.index.max()
        fc = model.forecast(steps=steps)
        for y in FORECAST_YEARS:
            ets_future_rows.append((field, y, float(fc.loc[y])))

    except Exception:
        last_val = float(series.iloc[-1])
        for y in FORECAST_YEARS:
            ets_future_rows.append((field, y, last_val))

ets_future = pd.DataFrame(ets_future_rows, columns=["ASJC_Field", "Year", "ETS_Pred"])

# Predict future residuals
future_resid = residual_model.predict(future_X[feature_cols_cat + feature_cols_num])

# Combine
future_pred = future_X[["ASJC_Field", "Year"]].copy()
future_pred = future_pred.merge(ets_future, on=["ASJC_Field", "Year"], how="left")
future_pred["Residual_Pred"] = future_resid
future_pred["Hybrid_Pred"] = future_pred["ETS_Pred"] + future_pred["Residual_Pred"]

# National forecast (sum across fields)
national_forecast = future_pred.groupby("Year")["Hybrid_Pred"].sum().reset_index()
national_forecast_df = national_forecast.copy()
print("\n=== National Publications Forecast (Hybrid) ===")
print(national_forecast)

# Optional: show field-level future predictions (top 10 rows)
print("\nSample field-level forecasts:")
print(future_pred.sort_values(["Year", "Hybrid_Pred"], ascending=[True, False]).head(10))

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
 


=== National Publications Forecast (Hybrid) ===
   Year  Hybrid_Pred
0  2026  6094.153189
1  2027  6094.153189
2  2028  6094.153189

Sample field-level forecasts:
                                 ASJC_Field  Year  ETS_Pred  Residual_Pred  \
0             Computer Science Applications  2026     583.0      -2.504815   
3                   Artificial Intelligence  2026     533.0     -12.537314   
6      Computer Networks and Communications  2026     441.0      -9.272217   
18  Computer Vision and Pattern Recognition  2026     434.0     -27.490613   
9     Electrical and Electronic Engineering  2026     352.0     -16.912065   
45                 Control and Optimization  2026     296.0       6.741575   
15         Civil and Structural Engineering  2026     306.0     -19.726817   
12       Information Systems and Management  2026     284.0     -14.369923   
30                          Instrumentation  2026     206.0      20.050875   
48            Waste Management and Disposal  2026     20

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [18]:
# =========================
# BUILD NATIONAL_ALL (Actual + Forecast) for dashboard
# =========================

# Actual totals from panel2 (2019–2024)
hist_actual = (
    panel2.groupby("Year")["Publications"]
    .sum()
    .reset_index()
)
hist_actual["Type"] = "Actual"

# Forecast totals from national_forecast_df (2026–2028)
forecast = national_forecast_df.copy()

# Ensure consistent column name
if "Hybrid_Pred" in forecast.columns:
    forecast = forecast.rename(columns={"Hybrid_Pred": "Publications"})
forecast["Type"] = "Forecast"

# Combine
national_all = pd.concat(
    [
        hist_actual[["Year", "Publications", "Type"]],
        forecast[["Year", "Publications", "Type"]],
    ],
    ignore_index=True
).sort_values("Year").reset_index(drop=True)

national_all.head(), national_all.tail()


(   Year  Publications    Type
 0  2019        4055.0  Actual
 1  2020        4544.0  Actual
 2  2021        6601.0  Actual
 3  2022        6155.0  Actual
 4  2023        6632.0  Actual,
    Year  Publications      Type
 4  2023   6632.000000    Actual
 5  2024   6563.000000    Actual
 6  2026   6094.153189  Forecast
 7  2027   6094.153189  Forecast
 8  2028   6094.153189  Forecast)

In [19]:
# =========================
# BUILD FIELD_FORECAST_DF (Field x Year forecasts)
# =========================

field_forecast_df = future_pred.copy()

# Keep only useful columns (safe even if some don't exist)
keep_cols = [c for c in [
    "ASJC_Field", "Year",
    "ETS_Pred", "Residual_Pred", "Hybrid_Pred"
] if c in field_forecast_df.columns]

field_forecast_df = field_forecast_df[keep_cols].copy()

field_forecast_df.head()


,ASJC_Field,Year,ETS_Pred,Residual_Pred,Hybrid_Pred
0,Computer Science Applications,2026,583.0,-2.504815,580.495185
1,Computer Science Applications,2027,583.0,-2.504815,580.495185
2,Computer Science Applications,2028,583.0,-2.504815,580.495185
3,Artificial Intelligence,2026,533.0,-12.537314,520.462686
4,Artificial Intelligence,2027,533.0,-12.537314,520.462686


In [20]:
eval_ws = sh.worksheet("Outputs_Evaluation")
nat_ws  = sh.worksheet("Outputs_National")
all_ws  = sh.worksheet("Outputs_National_AllYears")
fld_ws  = sh.worksheet("Outputs_Field_Forecast")

eval_ws.clear()
nat_ws.clear()
all_ws.clear()
fld_ws.clear()

set_with_dataframe(eval_ws, evaluation_df)
set_with_dataframe(nat_ws, national_forecast_df)
set_with_dataframe(all_ws, national_all)
set_with_dataframe(fld_ws, field_forecast_df)

print("✅ Dashboard data updated")


✅ Dashboard data updated


Connect with App Script

In [21]:
!pip -q install fastapi uvicorn pyngrok joblib


In [22]:
from pyngrok import ngrok

ngrok.set_auth_token("389SR6ZQlmroZV95KNdD8j1FecT_4cVDKUYtz4GCAf3KwoRxP")


In [23]:
from fastapi import FastAPI
import numpy as np
import pandas as pd

app = FastAPI()

# These must already exist in your notebook:
# - residual_model  (trained pipeline)
# - ets_future      (DataFrame with columns: ASJC_Field, Year, ETS_Pred)  OR a function to get ETS per field+year
# - last_feats_by_field (DataFrame/dict holding last-year feature values per field)
# - feature_cols_num, feature_cols_cat, top_fields (list of fields)

def _predict_one(field: str, year: int, feats: dict):
    # ETS lookup (per field-year)
    ets_val = float(
        ets_future.loc[(ets_future["ASJC_Field"] == field) & (ets_future["Year"] == year), "ETS_Pred"].iloc[0]
    )

    # Build ML feature row
    row = {"ASJC_Field": field, "Year": year}
    for k, v in feats.items():
        row[k] = v
    X = pd.DataFrame([row])

    resid = float(residual_model.predict(X[feature_cols_cat + feature_cols_num])[0])
    return ets_val, resid, ets_val + resid

@app.post("/predict")
def predict(data: dict):
    field = str(data.get("asjc_field", "")).strip()
    year  = int(data.get("year", 2027))

    # UI features (used only for single-field prediction)
    feats_ui = {
        "Citations": float(data.get("citations", 0)),
        "Field-Weighted Citation Impact": float(data.get("fwci", 0)),
        "Number of Authors": float(data.get("authors", 0)),
        "Number of Institutions": float(data.get("institutions", 0)),
        "Number of Countries/Regions": float(data.get("countries", 0)),
    }

    # ✅ NEW: handle All (national sum)
    if field.lower() == "all":
        ets_sum = 0.0
        resid_sum = 0.0
        hyb_sum = 0.0

        for f in top_fields:
            # Use your stored last-year features per field (recommended)
            # Example: last_feats_by_field is a dict {field: {feature: value}}
            feats = last_feats_by_field.get(f, feats_ui)  # fallback to ui feats if missing
            ets_v, resid_v, hyb_v = _predict_one(f, year, feats)
            ets_sum += ets_v
            resid_sum += resid_v
            hyb_sum += hyb_v

        return {
            "ETS_Pred": ets_sum,
            "Residual_Pred": resid_sum,
            "Hybrid_Pred": hyb_sum,
            "level": "national"
        }

    # Single field prediction (normal)
    ets_v, resid_v, hyb_v = _predict_one(field, year, feats_ui)
    return {
        "ETS_Pred": ets_v,
        "Residual_Pred": resid_v,
        "Hybrid_Pred": hyb_v,
        "level": "field",
        "field": field
    }



In [24]:
!pip -q install fastapi uvicorn pyngrok nest_asyncio

import nest_asyncio, threading, socket
import uvicorn
from pyngrok import ngrok

nest_asyncio.apply()

def get_free_port():
    s = socket.socket()
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

PORT = get_free_port()

def run():
    uvicorn.run(app, host="0.0.0.0", port=PORT, log_level="error")  # ✅ uses app from cell 23

ngrok.kill()
threading.Thread(target=run, daemon=True).start()

public_url = ngrok.connect(PORT).public_url
print("HEALTH URL :", public_url + "/health")
print("PREDICT URL:", public_url + "/predict")



HEALTH URL : https://sizy-merilyn-bombastic.ngrok-free.dev/health
PREDICT URL: https://sizy-merilyn-bombastic.ngrok-free.dev/predict


In [25]:
import requests

url = "https://sizy-merilyn-bombastic.ngrok-free.dev/predict"

payload = {
  "asjc_field": "Food Science",
  "year": 2027,
  "citations": 200,
  "fwci": 1.5,
  "authors": 5,
  "institutions": 3,
  "countries": 2
}

res = requests.post(url, json=payload)
print(res.status_code)
print(res.text)


200
{"ETS_Pred":110.0,"Residual_Pred":-19.737029761904765,"Hybrid_Pred":90.26297023809524,"level":"field","field":"Food Science"}
